In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from vocab import Vocabulary
from env import ConceptData
from create_data import addFile
import tensorflow as tf
import random
import numpy as np
import csv
import re
import os
import create_data
from xml.dom import minidom
import xml.etree.ElementTree as ET
from vocab import Vocabulary
from env2 import ConceptData
from create_data import addFile
import agents
import sender

In [ ]:
sender_encoder = sender.Sender(num_options=10, batch_size=32)
sender_only_target = sender.SenderOnlyTarget(batch_size=32)
sender = sender.Sender_LSTM(vocab_size=99, embed_dim=44, num_cells=1, hidden_size=1, max_len=10, see_all_input=True)
receiver_decoder = agents.Receiver(num_options=10)
receiver_encoder = agents.Receiver_LSTM(agent=receiver_decoder, vocab_size=99, embed_dim=44, hidden_size=44, masking_value=99)

In [ ]:
vocabulary = Vocabulary()

def addFile(name):
    file_name = name + "_structured_final.xml"
    file = minidom.parse(os.path.join(os.path.join('visa_dataset', 'UK'), file_name))
    concepts = file.getElementsByTagName('concept')

    for concept in concepts:
        vocabulary.addConcept(concept)


addFile("ANIMALS")
addFile("APPLIANCES")
addFile("ARTEFACTS")
addFile("CLOTHING")
addFile("CONTAINER")
addFile("DEVICE")
addFile("FOOD")
addFile("HOME")
addFile("INSTRUMENTS")
addFile("MATERIAL")
addFile("PLANTS")
addFile("STRUCTURES")
addFile("TOOLS")
addFile("TOYS")
addFile("VEHICLES")
addFile("WEAPONS")

for concept in vocabulary.concept_list:
    vocabulary.parseConcept(concept)
    
data = ConceptData(vocabulary, 9, 32)

In [ ]:
input_concepts, sender_input, targets, receiver_input = data.getInput()
#print(sender_input.shape)

In [ ]:
sender_encoded = sender_encoder(sender_input)
message, logits, entropy, state_h = sender(sender_encoded)
print("message:")
print(message)
print("logits:")
print(logits)
print("entropy:")
print(entropy)
print("state_h:")
print(state_h)

In [ ]:
sample, log_prob_r, entropy_r, last_hidden = receiver_encoder(message=message, batch_size=32, max_len=10, receiver_input=receiver_input)

receiver_output = []
for i in range(32):
    receiver_output.append(input_concepts[i][sample[i]])
    
print(receiver_output)